In [ ]:
python src/gpsro_utils/convert_bufr2ioda.py /discover/nobackup/projects/gmao/geos-it/mchattop/GPSRO_SPIRE_reanalysis/GPSRO_final/Y2022/M01
python src/gpsro_utils/convert_ioda2df.py wrkdir/function-outputs/iodadir/GPSRO_final/Y2022/M01/

files_ioda = get_file_list(rootdir)


In [ ]:
def display_help(func):
    """
    Displays the help message for a given function.

    Parameters:
        func (function): The function to display help for.
    """
    try:
        print(help(func))
    except TypeError:
        print("Error: Invalid function provided.")


# Example Usage
result = add_numbers(10, "5")  # This will print an error message
display_help(add_numbers)      # This will print the docstring of add_numbers

result = divide_numbers(10, 0)  # This will print an error message for division by zero
display_help(divide_numbers)    # This will print the docstring of divide_numbers

def calculate_average(numbers):
    """Calculates the average of a list of numbers.

    Args:
        numbers: A list of numerical values.

    Returns:
        The average of the numbers.

    Raises:
        ValueError: If the list is empty.
    """

    if not numbers:
        raise ValueError("Cannot calculate average of an empty list.")

    return sum(numbers) / len(numbers)

# Using the function
try:
    result = calculate_average([1, 2, 3, 4, 5])
    print("Average:", result)

    result = calculate_average([])
    print("Average:", result) 

except ValueError as e:
    print("Error:", e)

In [ ]:
def verify_input_type(cli_input):
            
    #my_file = Path(rootdir)
    rootdir=sys.argv[1]
    input1 = Path(sys.argv[1])

    if input1.is_file():
        # file exists
        files = [sys.argv[1]]
        print(f'input is a file: \n {files}')
    if input1.is_dir():
        # directory exists    
        files = get_file_list(sys.argv[1])
        print(f'input is a directory: \n {files}')
    


In [ ]:
from netCDF4 import Dataset

def get_meta_data(data):
    MetaData = data.groups['MetaData']
    print(MetaData)
    return(MetaData)
    
##pathbufr = '/discover/nobackup/projects/gmao/geos-it/mchattop/GPSRO_SPIRE_reanalysis/GPSRO_final/Y2022/M01/gdas1_spnasa.220101.t00z.gpsro.tm00.bufr_d'
##pathioda = 'wrkdir/function-outputs/iodadir/GPSRO_final/Y2022/M01/gdas1_spnasa.220101.t00z.gpsro.tm00.ioda.nc4'

pathioda = 'gdas1_spnasa.220101.t00z.gpsro.tm00.ioda.nc4'
pathbufr = 'gdas1_spnasa.220101.t00z.gpsro.tm00.bufr_d'
ioda = Dataset(pathioda, mode ='r')

meta = get_meta_data(ioda)

In [ ]:

import sys 
from netCDF4 import Dataset
import pandas as pd
import subprocess
from discover_config import gmao_utils_dir,GPSRO_SPIRE_reanalysis,BufrTableC,wrkdir,outdir,iodadir
from file_utils import get_file_list,merge_bufr_code_table_c
from convert_ioda2df import nc_to_df
from ioda_utils import get_meta_data,get_obs_value,get_obs_error,satellite_id_subset,lat_window_subset,get_variable
testfile = '/discover/nobackup/sicohen/workenv/gmao-utils/src/gpsro_utils/wrkdir/function-outputs/iodadir/GPSRO_final/Y2022/M01/gdas1_spnasa.220101.t00z.gpsro.tm00.ioda.nc4'



In [ ]:
# subset by kx and plot latitude 5 degree bins
import pandas as pd
from ioda_utils import *
import sys
import numpy as np 

from discover_config import gmao_utils_dir,GPSRO_SPIRE_reanalysis,BufrTableC,wrkdir,outdir,iodadir
from ioda_utils import get_meta_data,get_obs_value,get_obs_error,satellite_id_subset,lat_window_subset,get_variable

data=sys.argv[1]
## def get_variable(data,group,name):
dateTime = pd.to_datetime(get_variable(data,'MetaData','dateTime'),unit='s') 
latitude = get_variable(data,'MetaData','latitude')
longitude = get_variable(data,'MetaData','longitude')
bendingAngle = get_variable(data,'ObsValue','bendingAngle')
satelliteIdentifier = get_variable(data,'MetaData','satelliteIdentifier')

dtype = {'names' : ['dateTime', 'latitude', 'longitude', 'bendingAngle','satelliteIdentifier'],
         'formats' : ['datetime64[ns]','float32','float32','float32','int32']}

arr = np.rec.fromarrays([dateTime, latitude, longitude, bendingAngle, satelliteIdentifier], dtype = dtype)

#def satellite_id_subset(arr, kx):
#satellite_id_subset(arr, 269)

cosmic2 = [750,751,752,753,754,755]
cosmic = [740,741,742,743,744,745]
spire = [269]
metop = [4,5]
other = [42,43,44,786,800,803,804,820,825]
[cosmic,cosmic2,spire,metop,other]

c=0
colors = ['blue','orange','green','red','purple','black','cyan','pink','brown']
for kx in [cosmic,cosmic2,spire,metop,other]:
    c += 1
    print(kx)
    dt = satellite_id_subset(arr, kx)
    plt.hist(dt['latitude'], bins=len(np.arange(-90,90,5.)), color = colors[c])
    plt.xticks(np.arange(-90,90+10,10.))
    plt.xlabel('Latitude')
    plt.ylabel('Counts')
    plt.title(f'Latitude Counts (5 degree bins)  ~ kx: {kx}',
              fontweight = "bold")
    plt.show() 
    #subprocess.run(['mkdir', '-p', 'wrkdir/figures'])
    #plt.savefig('wrkdir/figures/Latitude_Counts_5_degree_bins_test.png')
    #plt.show() 

In [2]:
# plot latitude 5 degree bins 

import sys 
from netCDF4 import Dataset
import pandas as pd
import subprocess
from discover_config import gmao_utils_dir,GPSRO_SPIRE_reanalysis,BufrTableC,wrkdir,outdir,iodadir
from file_utils import get_file_list,merge_bufr_code_table_c
from convert_ioda2df import nc_to_df
from ioda_utils import get_meta_data,get_obs_value,get_obs_error,satellite_id_subset,lat_window_subset,get_variable


# ---- (Helper) Functions

def get_meta_data(data):
    MetaData = data.groups['MetaData']
    print(MetaData)
    return(MetaData)

def get_obs_value(data):
    ObsValue = data.groups['ObsValue']
    print(ObsValue)
    return(ObsValue)

def get_obs_error(data):
    ObsError = data.groups['ObsError']
    print(ObsError)
    return(ObsError)

def satellite_id_subset(arr, kx):
    arr_kx = arr[np.isin(arr['satelliteIdentifier'], kx)]
    #print(arr.dtype.names)
    return(arr_kx)

def lat_window_subset(arr, window):
    arr_kx = arr[np.isin(arr['latitude'], window)]
    return(arr_kx)

def get_variable(data,group,name):
    x = data.groups[group].variables[name][:]
    return(x)

#testfile = Dataset('wrkdir/function-outputs/iodadir/GPSRO_final/Y2022/M01/gdas1_spnasa.220101.t00z.gpsro.tm00.ioda.nc4', mode ='r')
#latitude = get_variable(testfile,'MetaData','latitude')

testfile = 'wrkdir/function-outputs/iodadir/GPSRO_final/Y2022/M01/gdas1_spnasa.220101.t00z.gpsro.tm00.ioda.nc4'
filepath = testfile 
("_").join(fileparts[0:3])

filepath = sys.argv[1]
filename = filepath.split("/")[-1]
fileparts = filename.split(".")
"latitude_counts_" + ("_").join(fileparts[0:3]) + ".png" 

# Plot ~ Latitude Counts (5 degree bins)
import matplotlib.pyplot as plt
import numpy as np
import sys
import subprocess

# data
#testfile = Dataset('wrkdir/function-outputs/iodadir/GPSRO_final/Y2022/M01/gdas1_spnasa.220101.t00z.gpsro.tm00.ioda.nc4', mode ='r')
ioda = Dataset(sys.argv[1], mode ='r')
latitude = get_variable(ioda,'MetaData','latitude')

#  plot data ----- 
plt.hist(latitude,bins=len(np.arange(-90,90,5.)))

# graph options
plt.xlabel('Latitude Bins (5 degrees)')
plt.ylabel('Counts')
plt.title('Latitude Counts (5 degree bins)',
          fontweight = "bold")

subprocess.run(['mkdir', '-p', 'wrkdir/figures'])
plt.savefig('wrkdir/figures/Latitude_Counts_5_degree_bins_test.png')

plt.show() 



SyntaxError: invalid syntax (3127179325.py, line 50)

In [1]:
python src/gpsro_utils/plot_latitude_bins.py wrkdir/function-outputs/iodadir/GPSRO_final/Y2022/M01/gdas1_spnasa.220101.t00z.gpsro.tm00.ioda.nc4


SyntaxError: invalid syntax (2059146717.py, line 1)

In [ ]:
from discover_config import gmao_utils_dir,GPSRO_SPIRE_reanalysis,BufrTableC,wrkdir,outdir,iodadir
def merge_bufr_code_table_c(dataframe):
    # Merge the Bufr SAID codes ~ add names for satid's
    BufrCodes = pd.read_csv('BufrTableC')
    BufrCodes['SAID'] = BufrCodes['SAID'].astype(float).astype(int)
    BufrCodes['kx'] = BufrCodes['SAID'].abs().astype(int)
    dataframe['kx'] = dataframe['satid'].astype(int)
    dataframe = dataframe.merge(BufrCodes, how = 'outer', on = 'kx')
    dataframe = dataframe[~dataframe['bendingAngle'].isna()]
    #print(df)
    #print(df.columns)
    return(dataframe)

    
    #rootdir=df.filepath[0]
#df = df.sort_values(by=['DATE','kx'])

##############################
#saved_dataframe_path = '/discover/nobackup/sicohen/Data/gmao/ioda/GPSRO/Y2009/M06/GPSRO.Y2009.M05.ioda_nc4.csv'
#df.to_csv(saved_dataframe_path, index=False)

In [ ]:
import sys
from netCDF4 import Dataset
import pandas as pd
import subprocess
from discover_config import gmao_utils_dir,GPSRO_SPIRE_reanalysis,BufrTableC,wrkdir,outdir,iodadir
from file_utils import merge_bufr_code_table_c
#data = nc.Dataset("gdas1_spnasa.220101.t00z.gpsro.tm00.ioda.nc4", mode='r')
files_ioda = ["gdas1_spnasa.220101.t00z.gpsro.tm00.ioda.nc4"]

def nc_to_df(files_ioda):
    df_all = pd.DataFrame([], columns = ['dateTime', 'latitude', 'longitude', 'bendingAngle','impp',
                                    'satid','seq', 'satConstRO', 'satTransId','geoidU', 'refr',
                                    'filename','filepath'])
    j=0
    for i in files_ioda:
        j+=1
        print(f'Converting (.nc4 to pandas Dataframe) file {j} out of {len(files_ioda)}: {i}')
        data = Dataset(i, mode='r')
        df = pd.DataFrame(
            {
                'dateTime': data.groups['MetaData'].variables['dateTime'][:], 
                'latitude': data.groups['MetaData'].variables['latitude'][:],
                'longitude': data.groups['MetaData'].variables['longitude'][:],
                'bendingAngle': data.groups['ObsValue'].variables['bendingAngle'][:],
                'impp': data.groups['MetaData'].variables['impactParameterRO'][:],
                'satid': data.groups['MetaData'].variables['satelliteIdentifier'][:],
                'seq': data.groups['MetaData'].variables['sequenceNumber'][:],
                'satConstRO': data.groups['MetaData'].variables['satelliteConstellationRO'][:],
                'satTransId': data.groups['MetaData'].variables['satelliteTransmitterId'][:],
                'geoidU': data.groups['MetaData'].variables['geoidUndulation'][:],
                'refr': data.groups['ObsValue'].variables['atmosphericRefractivity'][:],
                'filename':i.split("/")[-1],
                'filepath':i,
                'filedate':i.split(".")[1],
                'filecycle':i.split(".")[2],
                'filedatetime':i.split(".")[1] + i.split(".")[2],
                'DATE': pd.to_datetime(data.groups['MetaData'].variables['dateTime'][:],unit='s') 
                        #pd.to_datetime(df['dateTime'],unit='s')
            }
        )
        #print(df)
        print(i)
        df_all = pd.concat([df_all,df])
        print(f' Progress: {round(j/len(files_ioda)*100)}%')
        return(df_all)



df = nc_to_df(files_ioda)

In [ ]:
from discover_config import gmao_utils_dir,GPSRO_SPIRE_reanalysis,BufrTableC,wrkdir,outdir,iodadir
# Merge the Bufr SAID codes ~ add names for satid's
############################################################
BufrCodes = pd.read_csv(BufrTableC)
BufrCodes['SAID'] = BufrCodes['SAID'].astype(float).astype(int)
BufrCodes['kx'] = BufrCodes['SAID'].abs().astype(int)
df['kx'] = df['satid'].astype(int)
df = df.merge(BufrCodes, how = 'outer', on = 'kx')
df = df[~df['bendingAngle'].isna()]
print(df)
print(df.columns)
##############################

In [ ]:
############################################################################################################
# 2024/03/28
# convert a directory of converted ioda.nc4 GPSRO n files to pandas dataframe

import pandas as pd
from netCDF4 import Dataset
import sys
from GetInputFileListEnumerate import *
import subprocess

ioda_path = '/discover/nobackup/sicohen/Data/gmao/ioda/GPSRO_final/Y2022/M01'
saved_dataframe_path = '/discover/nobackup/sicohen/Data/gmao/ioda/GPSRO_final/Y2022/GPSRO.SPNASA.Y2022.M01.ioda_nc4.csv'
plot_filename = '/discover/nobackup/sicohen/Data/gmao/ioda/GPSRO_final/figures/GPSRO.SPNASA.Y2022.M01.ioda_nc4.p1.counts.png'


####################################
#def iodanc4_to_dataframe(files_ioda):
####################################
q = []
df_all = pd.DataFrame(q, columns = ['dateTime', 'latitude', 'longitude', 'bendingAngle','impp',
                                    'satid','seq', 'satConstRO', 'satTransId','geoidU', 'refr',
                                    'filename','filepath'])



# 10/24/2023
# GetInputFileListEnumerate() ----------------------------------------------------------------
import os
import sys
    # path = '/discover/nobackup/sicohen/Data/gmao/bufr/AMSUA/Y2020/'
    # rootdir = sys.argv[1]
    # 'filelist' with be the list of bufr files that is returned (to then be used as input for the inventory tool)
    # loop over directory and all it's subdirectories, list whats in there but dont print everything, return only bufr files account for files name with extensions '.bufr' and '.bufr_d'
def GetInputFileListEnumerate(rootdir):#,suffix):
    filelist = []
    for subdir, dirs, files in os.walk(rootdir):
        print(f"{dirs}")
        print(f"----{subdir}")
        for idx, file in enumerate(files):
            if file.endswith("ioda.nc4"): # or file.endswith(".bufr_d"):
                print (f"--------- {os.path.join(subdir, file)}")
                filepath = subdir + os.sep + file
                filelist.append(filepath)
    return(sorted(filelist))    #print(sorted(filelist))

# list of files 
files_ioda = GetInputFileListEnumerate(ioda_path)


j=0
for i in files_ioda:
    j+=1
    print(f'Converting (.nc4 to pandas Dataframe) file {j} out of {len(files_ioda)}: {i}')
    data = Dataset(i, mode='r')
    df = pd.DataFrame(
        {
            'dateTime': data.groups['MetaData'].variables['dateTime'][:], 
            'latitude': data.groups['MetaData'].variables['latitude'][:],
            'longitude': data.groups['MetaData'].variables['longitude'][:],
            'bendingAngle': data.groups['ObsValue'].variables['bendingAngle'][:],
            'impp': data.groups['MetaData'].variables['impactParameterRO'][:],
            'satid': data.groups['MetaData'].variables['satelliteIdentifier'][:],
            'seq': data.groups['MetaData'].variables['sequenceNumber'][:],
            'satConstRO': data.groups['MetaData'].variables['satelliteConstellationRO'][:],
            'satTransId': data.groups['MetaData'].variables['satelliteTransmitterId'][:],
            'geoidU': data.groups['MetaData'].variables['geoidUndulation'][:],
            'refr': data.groups['ObsValue'].variables['atmosphericRefractivity'][:],
            'filename':i.split("/")[-1],
            'filepath':i,
            'filedate':i.split(".")[1],
            'filecycle':i.split(".")[2],
            'filedatetime':i.split(".")[1] + i.split(".")[2],
            'DATE': pd.to_datetime(data.groups['MetaData'].variables['dateTime'][:],unit='s') 
                    #pd.to_datetime(df['dateTime'],unit='s')
        }
    )
    #print(df)
    print(i)
    df_all = pd.concat([df_all,df])
    print(f' Progress: {round(j/len(files_ioda)*100)}%')
    #return df_all
####################################

df_all.sort_values(by=['filename','DATE'])
df=df_all


# 
#df['dateTime'] = pd.to_datetime(df['dateTime'], format='s')
#df['DATE'] = pd.to_datetime(df['dateTime'],unit='s')

# Merge the Bufr SAID codes ~ add names for satid's
############################################################
BufrCodes = pd.read_csv('/discover/nobackup/sicohen/Workshop/SicInventoryTools/dependencies/bufr-code-table-C-sinv-merge.csv')
BufrCodes['SAID'] = BufrCodes['SAID'].astype(float).astype(int)
BufrCodes['kx'] = BufrCodes['SAID'].abs().astype(int)
df['kx'] = df['satid'].astype(int)
df = df.merge(BufrCodes, how = 'outer', on = 'kx')
df = df[~df['bendingAngle'].isna()]
print(df)
print(df.columns)
##############################

#rootdir=df.filepath[0]
#df = df.sort_values(by=['DATE','kx'])

##############################
#saved_dataframe_path = '/discover/nobackup/sicohen/Data/gmao/ioda/GPSRO/Y2009/M06/GPSRO.Y2009.M05.ioda_nc4.csv'
df.to_csv(saved_dataframe_path, index=False)
